In [1]:
from mpl_toolkits.basemap import Basemap  # import Basemap matplotlib toolkit
import numpy as np
import matplotlib.pyplot as plt
import pygrib # import pygrib interface to grib_api
import pandas as pd
import os
import sys
import math
import folium
import csv 

%matplotlib inline

root = os.getcwd()
root_data = root + "/grib_data"
root_URL = "http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/"
year = "2018"
month = "08"
day = "01"

def getgrib(year,month,day):
    file_name = "Z__C_RJTD_"+str(year+month+day)+"000000_MSM_GPV_Rjp_Lsurf_FH00-15_grib2.bin"
    file_url = root_URL\
                +str(year + "/" + month + "/" + day +"/" )\
                +file_name
    root_file= "grib_data/"+file_name
    if os.path.isfile(root_file) == False:
            command = "wget -P grib_data/ "+ file_url
            os.system(command)

    getgrib_output = pygrib.open(root_file)
    return getgrib_output

def getrootgrib(year,month,day):
    file_name = "Z__C_RJTD_"+str(year+month+day)+"000000_MSM_GPV_Rjp_Lsurf_FH00-15_grib2.bin"    
    return "grib_data/"+file_name

In [2]:
print(os.path.isfile( getrootgrib(year,month,day) ))

True


In [3]:
grbs = getgrib(year,month,day)

In [4]:
print(os.path.isfile( getrootgrib(year,month,day) ))

True


In [5]:
#check parameters in gribfile
for grb in grbs:
    print(grb)

1:Pressure reduced to MSL:Pa (instant):regular_ll:meanSea:level 0:fcst time 0 hrs:from 201808010000
2:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201808010000
3:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 0 hrs:from 201808010000
4:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 0 hrs:from 201808010000
5:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 0 hrs:from 201808010000
6:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 0 hrs:from 201808010000
7:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201808010000
8:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201808010000
9:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201808010000
10:Total cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 2

121:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 10 hrs:from 201808010000
122:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 10 hrs:from 201808010000
123:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 10 hrs:from 201808010000
124:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 10 hrs:from 201808010000
125:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 10 hrs:from 201808010000
126:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 10 hrs:from 201808010000
127:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 10 hrs:from 201808010000
128:Total cloud cover:% (instant):regular_ll:surface:level 0:fcst time 10 hrs:from 201808010000
129:Total precipitation:kg m-2 (accum):regular_ll:surface:level 0:fcst time 9-10 hrs (accum):from 201808010000
130:Downward short-wave radiation flux:W m**-2 

In [6]:
# lats,lonsは二次元配列で緯度経度が入っている
lats, lons = grb.latlons()

#一次元に変換
# 二次元のままでは描画できない
# 2次元の格子点を1次元配列にならす
flat_lats= np.ravel(lats)
flat_lons= np.ravel(lons)

pd_all= {'lons':flat_lons, 'lats':flat_lats}

#ID 27 is Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 2 hrs:from 201801010000
# selectメソッドで指定した要素(forecastTime=0)にマッチする要素の配列が返ってくる。
# forecastTime=0の要素は複数あるが先頭([0])はTempratureが入っている

for i in range(0,6):
    grb = grbs.select(forecastTime=i)[5]
    # 中に含まれているデータ(temprature)を二次元のndarrayで取り出す
    temperature = grb.values
    #華氏を摂氏に変換
    flat_temperature= (np.ravel(temperature)-32)/1.8
    pd_name="temperature FH"+str(i)
    pd_all[pd_name]=flat_temperature
    
df_all= pd.DataFrame(pd_all)
print(len(df_all))
df_all[:10]

242905


,lons,lats,temperature FH0,temperature FH1,temperature FH2,temperature FH3,temperature FH4,temperature FH5
0,120.0000,47.6,18.279455,15.649609,12.724281,10.939543,9.564295,8.538890
1,120.0625,47.6,18.088483,15.441276,12.672198,10.904821,9.546934,8.538890
2,120.1250,47.6,18.019038,14.955165,12.168725,10.383987,9.060823,8.104863
3,120.1875,47.6,17.966955,14.712109,11.734698,9.880515,8.609434,7.705557
4,120.2500,47.6,18.262094,14.764192,11.630531,9.776349,8.522628,7.636113
5,120.3125,47.6,18.348899,14.729470,11.526364,9.654821,8.383739,7.497224
6,120.3750,47.6,18.088483,14.677387,11.474281,9.463849,8.140684,7.254168
7,120.4375,47.6,17.949594,14.816276,11.665253,9.550654,8.175406,7.219446
8,120.5000,47.6,17.949594,14.781553,11.752059,9.706904,8.314295,7.306252
9,120.5625,47.6,17.064177,13.739887,11.005531,8.977737,7.585128,6.542363


In [7]:
#緯度経度の範囲を選択
east_lon=131.69
west_lon=129.4337
north_lat=34.1413
south_lat=31.01

In [8]:
#range adjust
df=df_all[df_all['lons'] > west_lon]
df=df[df['lons'] < east_lon]

df=df[df['lats'] > south_lat]
df=df[df['lats'] < north_lat]
print(len(df))
df[:10]

2294


,lons,lats,temperature FH0,temperature FH1,temperature FH2,temperature FH3,temperature FH4,temperature FH5
130021,129.4375,34.1,24.841955,26.187803,26.404836,26.859682,26.612906,26.264585
130022,129.5000,34.1,25.015566,26.326692,26.404836,26.929126,26.578184,26.229863
130023,129.5625,34.1,25.189177,26.396137,26.422198,26.911765,26.526100,26.177779
130024,129.6250,34.1,25.362788,26.430859,26.456920,26.842321,26.456656,26.160418
130025,129.6875,34.1,25.519038,26.430859,26.509003,26.772876,26.369850,26.177779
130026,129.7500,34.1,25.692649,26.413498,26.578448,26.686071,26.300406,26.177779
130027,129.8125,34.1,25.831538,26.396137,26.630531,26.581904,26.213600,26.212502
130028,129.8750,34.1,25.970427,26.413498,26.682614,26.477737,26.092073,26.247224
130029,129.9375,34.1,26.039872,26.448220,26.682614,26.356210,25.987906,26.247224
130030,130.0000,34.1,25.987788,26.500303,26.630531,26.234682,25.953184,26.177779


In [9]:
map_lat=df['lats'].values
map_lon=df['lons'].values
FH0=np.round(df['temperature FH0'].values,2)
FH1=np.round(df['temperature FH1'].values,2)
FH2=np.round(df['temperature FH2'].values,2)
FH3=np.round(df['temperature FH3'].values,2)
FH4=np.round(df['temperature FH4'].values,2)
FH5=np.round(df['temperature FH5'].values,2)

In [10]:
from folium import plugins
#lat-long of focus area and zoom start
map = folium.Map([33.518022,130.471583], zoom_start=12)

# Mapbox makes some nice map tiles,
# Stamen also produce some cool map tiles which typically work at all zoom levels.
tile = folium.TileLayer('Stamen Terrain').add_to(map)

#adding marker and popup of temperature
for i in range(0,len(map_lat)):
    
    icon_number = plugins.BeautifyIcon(
    border_color='#00ABDC',
    text_color='#00ABDC',
    border_width=2,
    number=str(int(FH0[i])),
    inner_icon_style='margin-top:0;')
    
    folium.Marker(location=[float(map_lat[i])  ,float(map_lon[i])],\
                  popup="___temperature___"+"\n"\
                          +"__FH0= "+str(FH0[i])+"__\n"\
                          +"__FH1 = "+str(FH1[i])+"__\n"\
                          +"__FH2 = "+str(FH2[i])+"__\n"\
                          +"__FH3 = "+str(FH3[i])+"__\n"\
                          +"__FH4 = "+str(FH4[i])+"__\n"\
                          +"__FH5 = "+str(FH5[i])+"__\n",\
                 icon=icon_number).add_to(map)
    
#we can change tiles with help of LayerConrol
folium.LayerControl().add_to(map)

map_name="Output_MAP/"+"FH_Temperature Map: " + year + "_" + month + "_" + day+".html"
map.save(outfile=map_name)
map